In [1]:
from math import sqrt
import numpy as np
import matplotlib.path as mpl_path
import matplotlib.pyplot as plt
import pre_processing as pp
import csv
import pandas as pd
import tensorflow as tf
from point_grid import *

In [ ]:
#==========================
#   EXPLORE DATASET
#==========================
df = pd.read_csv('data/8-gon-correct-procrustes.csv')
df = df[df.internal_nodes == 3.0].dropna(axis=1, how='all')
df = df[df.target_edge_length == 0.4]
dataset = df.drop(['target_edge_length', 'internal_nodes'], axis=1)

# Patch dataset generation

In [4]:
internal_count = 3
edge_count = 8


df = pd.read_csv('data/8-gon-correct-procrustes.csv')
df = df[df.internal_nodes == 3.0].dropna(axis=1, how='all')
df = df[df.target_edge_length == 0.4]
dataset = df.drop(['target_edge_length', 'internal_nodes'], axis=1)

# Csv-file to write to.
new_csv_path = "data/8-gon-patch-data-gmsh-3-internal.csv"
generate_patch_collection_and_write_to_csv(dataset, new_csv_path, edge_count=edge_count,internal_count=internal_count)

# Neural network 2: predicting internal node positions (or rather: predict grid score)

- 8 edges
- 3 internal nodes

In [12]:
# ======================
#        DATASET 
# ======================
import pandas as pd

patch_path = 'data/8-gon-patch-data-gmsh-3-internal.csv'
dataset = pd.read_csv(patch_path)

# Split dataset into 70/15/15 training/validation/test
patch_train = dataset.sample(frac=0.85)
patch_test = dataset.drop(patch_train.index)

# Split dataset into features and labels; last 4 (grid scores)
train_features = patch_train.iloc[:, :-4]
train_labels = patch_train.iloc[:, -4:]

test_features = patch_test.iloc[:, :-4]
test_labels = patch_test.iloc[:, -4:]

In [13]:
# packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import os

# ======================
#       MODEL STUFF
# ======================
INITIAL_LEARNING_RATE = 1e-3
EPOCHS = 5000
BATCH_SIZE = 512

model_path = 'model/thesis-grid-8gon-3int'

def grid_model_setup(edge_count=8):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(edge_count*2+8,)),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(edge_count*2+4, activation='relu'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(edge_count*2+4, activation='relu'),
        tf.keras.layers.BatchNormalization(),

        tf.keras.layers.Dense(4),
    ])

    return model

model = grid_model_setup()
model.summary()

decay_steps= 10_000
lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(1e-1, decay_steps, 1e-3)

model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(
                  learning_rate=lr_schedule,
              ),
              )

checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=EPOCHS//10, min_delta=0.0001)


history = model.fit(train_features,
                    train_labels,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    validation_split=0.18,
                    verbose=2,
                    callbacks=[checkpoint, early_stopping],
                    )

# ======================
#       EVALUATION
# ======================
train_acc = model.evaluate(
    train_features, train_labels, verbose=0)
test_acc = model.evaluate(
    test_features, test_labels, verbose=0)
print('Training data loss: %.3f, Test data loss: %.3f' %
        (train_acc, test_acc))


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 20)                80        
_________________________________________________________________
dense_3 (Dense)              (None, 20)                420       
_________________________________________________________________
batch_normalization_4 (Batch (None, 20)                80        
_________________________________________________________________
dense_4 (Dense)              (None, 20)                420       
_________________________________________________________________
batch_normalization_5 (Batch (None, 20)                80        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 84        
Total params: 1,164
Trainable params: 1,044
Non-trainable params: 120
__________________________________________________

ValueError: in user code:

    c:\Users\mkunn\skole\master\venv\masterMLvenv\lib\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    c:\Users\mkunn\skole\master\venv\masterMLvenv\lib\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\mkunn\skole\master\venv\masterMLvenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\mkunn\skole\master\venv\masterMLvenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\mkunn\skole\master\venv\masterMLvenv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
   